In [13]:
import boto3
import pandas as pd

In [14]:
SMART_MONEY_TYPES = ['accelerator', 'entrepreneurship_program', 'incubator', 'startup_competition']

In [15]:
cb_organisations = pd.read_parquet('organizations.parquet')
investors = pd.read_parquet('investors.parquet')

In [42]:
investments = pd.read_parquet('investments.parquet')

In [40]:
cb_organisations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3302538 entries, 0 to 3302537
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   id                           object 
 1   name                         object 
 2   type                         object 
 3   permalink                    object 
 4   cb_url                       object 
 5   rank                         float64
 6   created_at                   object 
 7   updated_at                   object 
 8   legal_name                   object 
 9   roles                        object 
 10  domain                       object 
 11  homepage_url                 object 
 12  country_code                 object 
 13  state_code                   object 
 14  region                       object 
 15  city                         object 
 16  address                      object 
 17  postal_code                  object 
 18  status                       object 
 19  

In [25]:
unique_investor_types = list(investors.investor_types.unique())
# Go through each and split by comma
# remove none
unique_investor_types = [x for x in unique_investor_types if x is not None]
investor_types = []
for investor_type in unique_investor_types:
    investor_types += investor_type.split(',')
# Remove any leading/trailing whitespace
investor_types = [x.strip() for x in investor_types]
# keep only unique values
investor_types = list(set(investor_types))

sorted(investor_types)


['accelerator',
 'angel',
 'angel_group',
 'co_working_space',
 'corporate_venture_capital',
 'entrepreneurship_program',
 'family_investment_office',
 'fund_of_funds',
 'government_office',
 'hedge_fund',
 'incubator',
 'investment_bank',
 'investment_partner',
 'micro_vc',
 'pension_funds',
 'private_equity_firm',
 'secondary_purchaser',
 'startup_competition',
 'syndicate',
 'university_program',
 'venture_capital',
 'venture_debt']

In [29]:
funds = pd.read_parquet('funds.parquet')
funding_rounds = pd.read_parquet('funding_rounds.parquet')


In [17]:
funds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23644 entries, 0 to 23643
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           23644 non-null  object 
 1   name                         23644 non-null  object 
 2   type                         23644 non-null  object 
 3   permalink                    23644 non-null  object 
 4   cb_url                       23644 non-null  object 
 5   rank                         0 non-null      float64
 6   created_at                   23644 non-null  object 
 7   updated_at                   23644 non-null  object 
 8   entity_id                    23644 non-null  object 
 9   entity_name                  23644 non-null  object 
 10  entity_type                  23644 non-null  object 
 11  announced_on                 22660 non-null  object 
 12  raised_amount_usd            19467 non-null  float64
 13  raised_amount   

In [16]:
funding_rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625332 entries, 0 to 625331
Data columns (total 24 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  625332 non-null  object 
 1   name                                625332 non-null  object 
 2   type                                625332 non-null  object 
 3   permalink                           625332 non-null  object 
 4   cb_url                              625332 non-null  object 
 5   rank                                625289 non-null  float64
 6   created_at                          625332 non-null  object 
 7   updated_at                          625332 non-null  object 
 8   country_code                        619177 non-null  object 
 9   state_code                          318053 non-null  object 
 10  region                              619177 non-null  object 
 11  city                      

In [31]:
from currency_converter import CurrencyConverter
import numpy as np

def convert_currency(
    funding_df: pd.DataFrame,
    date_column: str,
    amount_column: str,
    currency_column: str,
    converted_column: str = None,
    target_currency: str = "GBP",
) -> pd.DataFrame:
    """
    Convert amount in any currency to a target currency using CurrencyConverter package.
    Deal dates should be provided in the datetime.date format
    NB: Rate conversion for dates before year 2000 is not reliable and hence
    is not carried out (the function returns nulls instead)

    Args:
        funding_df: A dataframe which must have a column for a date and amount to be converted
        date_column: Name of column with deal dates
        amount_column: Name of column with the amounts in the original currency
        currency_column: Name of column with the currency codes (eg, 'USD', 'EUR' etc)
        converted_column: Name for new column with the converted amounts

    Returns:
        Same dataframe with an extra column for the converted amount

    """
    # Column name
    converted_column = (
        f"{amount_column}_{target_currency}"
        if converted_column is None
        else converted_column
    )
    # Check if there is anything to convert
    rounds_with_funding = len(funding_df[-funding_df[amount_column].isnull()])
    df = funding_df.copy()
    if rounds_with_funding > 0:
        # Set up the currency converter
        Converter = CurrencyConverter(
            fallback_on_missing_rate=True,
            fallback_on_missing_rate_method="linear_interpolation",
            # If the date is out of bounds (eg, too recent)
            # then use the closest date available
            fallback_on_wrong_date=True,
        )
        # Convert currencies
        converted_amounts = []
        for _, row in df.iterrows():
            # Only convert deals after year 1999
            if (row[date_column].year >= 2000) and (
                row[currency_column] in Converter.currencies
            ):
                converted_amounts.append(
                    Converter.convert(
                        row[amount_column],
                        row[currency_column],
                        target_currency,
                        date=row[date_column],
                    )
                )
            else:
                converted_amounts.append(np.nan)
        df[converted_column] = converted_amounts
        # For deals that were originally in the target currency, use the database values
        deals_in_target_currency = df[currency_column] == target_currency
        df.loc[deals_in_target_currency, converted_column] = df.loc[
            deals_in_target_currency, amount_column
        ].copy()
    else:
        # If nothing to convert, copy the values and return
        df[converted_column] = df[amount_column].copy()
    return df

In [27]:
funding_rounds.head(3)

,id,name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,...,raised_amount_usd,raised_amount,raised_amount_currency_code,post_money_valuation_usd,post_money_valuation,post_money_valuation_currency_code,investor_count,org_id,org_name,lead_investor_ids
0,8a945939-18e0-cc9d-27b9-bf33817b2818,Angel Round - Meta,funding_round,facebook-angel--8a945939,https://www.crunchbase.com/funding_round/faceb...,258122.0,2007-05-27 06:08:18,2018-02-12 23:05:39,USA,CA,...,500000.0,500000.0,USD,NaN,NaN,None,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,3f47be49-2e32-8118-01a0-31685a4d0fd7
1,d950d7a5-79ff-fb93-ca87-13386b0e2feb,Series A - Meta,funding_round,facebook-series-a--d950d7a5,https://www.crunchbase.com/funding_round/faceb...,365093.0,2007-05-27 06:09:10,2018-02-12 23:52:16,USA,CA,...,12700000.0,12700000.0,USD,98000000.0,98000000.0,USD,4.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,b08efc27-da40-505a-6f9d-c9e14247bf36
2,6fae3958-a001-27c0-fb7e-666266aedd78,Series B - Meta,funding_round,facebook-series-b--6fae3958,https://www.crunchbase.com/funding_round/faceb...,509772.0,2007-05-27 06:09:36,2018-02-12 23:30:46,USA,CA,...,27500000.0,27500000.0,USD,502500000.0,502500000.0,USD,5.0,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,"e2006571-6b7a-e477-002a-f7014f48a7e3,8d5c7e48-..."


In [98]:
# import datetime
# from typing import Iterator

# class CrunchbaseEnricher:
#     """
#     Enrich Crunchbase organisation and funding round data
#     """

#     def __init__(self, organisation_ids: Iterator[str] = None):
#         """
#         Args:
#             cb_data_path: Optional argument to specify the location of the
#                 Crunchbase data, if it is different from the default location;
#                 NB: Note that this changes a global variable.
#         """
#         # Tables from the database
#         self.organisation_ids = organisation_ids
#         self._organizations = None
#         self._organizations_enriched = None
#         self._funding_rounds = None
#         self._funding_rounds_enriched = None

#     @property
#     def organizations(self):
#         """Full table of companies (this might take a minute to load in)"""
#         if self._organizations is None:
#             self._organizations = cb_organisations.head(10).copy() # change to getters
#         return self._organizations

#     @property
#     def funding_rounds(self):
#         """Table with investment rounds (all the deals for all companies)"""
#         if self._funding_rounds is None:
#             self._funding_rounds = funding_rounds.head(10).copy() # change to getters 
#         return self._funding_rounds

#     @property
#     def funding_rounds_enriched(self):
#         """Table with investment rounds (all the deals for all companies)"""
#         if self._funding_rounds_enriched is None:
#             self._funding_rounds_enriched = self.enrich_cb_funding_rounds(self.funding_rounds)
#         return self._funding_rounds

#     def enrich_cb_funding_rounds(
#         self,
#         funding_round_ids: Iterator[str] = None, 
#         cutoff_year: int = 1990,
#     ) -> pd.DataFrame:
#         """
#         Add funding round information to a table with CB organisations

#         Args:
#             cb_organisations: Data frame that must have a columns with crunchbase
#                 organisation ids and names

#         Returns:
#             Dataframe with organisations specified by 'org_id' and 'name', and
#             all their funding rounds (deals). Some of the important columns include:
#                 'funding_round_id': unique round id
#                 'announced_on': date of the round
#                 'investment_type': the type of round (seed, series etc.)
#                 'post_money_valuation_usd': valuation of the company
#                 'raised_amount_usd': amount of money raised in the deal (in thousands)

#         """
#         # If no IDs are specified, assume all funding rounds are needed
#         if funding_round_ids is None:
#             funding_round_ids = self.funding_rounds.id.unique()

#         return (
#             self.funding_rounds.query("id in @funding_round_ids")
#             # More informative column names
#             .rename(columns={"id": "funding_round_id"})
#             # Remove really old funding rounds
#             .query(f"announced_on > '{cutoff_year}'")
#             .sort_values("announced_on")
#             .assign(
#                 # Convert investment amounts to thousands
#                 raised_amount=lambda df: df["raised_amount"] / 1e+3,
#                 raised_amount_usd=lambda df: df["raised_amount_usd"] / 1e+3,
#                 # Convert date strings to datetimes
#                 announced_on_date=lambda df: pd.to_datetime(df["announced_on"]),
#             )
#             # Get years from dates
#             .assign(year=lambda df: df["announced_on_date"].apply(lambda x: x.year))
#             # Convert USD currency to GBP
#             .pipe(
#                 convert_currency,
#                 date_column="announced_on_date",
#                 amount_column="raised_amount",
#                 currency_column="raised_amount_currency_code",
#                 converted_column="raised_amount_gbp",
#                 target_currency="GBP",
#             )
#         )


#     def _process_total_funding_gbp(
#         funding_rounds_enriched_df: pd.DataFrame,
#         organisation_ids: Iterator[str] = None,
#         date_start: int = None,
#         date_end:  str = None,
#     ):
#         """
#         Get the total funding in GBP for a set of organisations

#         Args:
#             funding_rounds_enriched_df: Dataframe with enriched funding rounds
#             date_start: Earliest date for the funding rounds to include, in the format 'YYYY-MM-DD'
#             date_end: Latest date for the funding rounds to include, in the format 'YYYY-MM-DD'
        
#         Returns:
#             Dataframe with columns 'org_id' and 'total_funding_gbp'. Organisations that have no funding
#             in the specified time period will not be included in the dataframe
#         """
#         # If no IDs are specified, assume all organisations are needed
#         organisation_ids = organisation_ids or funding_rounds_enriched_df.org_id.unique()

#         # Determine the dates to include
#         date_start = date_start or funding_rounds_enriched_df.announced_on.min()
#         date_end = date_end or funding_rounds_enriched_df.announced_on.max()

#         return (
#             funding_rounds_enriched_df
#             .loc[funding_rounds_enriched_df['announced_on'].between(date_start, date_end, inclusive='both')]
#             .query("org_id in @organisation_ids")
#             .groupby("org_id")
#             .agg(total_funding_gbp=("raised_amount_gbp", "sum"))
#             .reindex(organisation_ids)
#             .reset_index()
#             .fillna(0)        
#         )



In [35]:
from src.utils import google
from typing import Iterator

def enrich_cb_funding_rounds(
    funding_rounds: pd.DataFrame,
    funding_round_ids: Iterator[str] = None, 
    cutoff_year: int = 2000,
) -> pd.DataFrame:
    """
    Add funding round information to a table with CB organisations

    Args:
        cb_organisations: Data frame that must have a columns with crunchbase
            organisation ids and names

    Returns:
        Dataframe with organisations specified by 'org_id' and 'name', and
        all their funding rounds (deals). Some of the important columns include:
            'funding_round_id': unique round id
            'announced_on': date of the round
            'investment_type': the type of round (seed, series etc.)
            'post_money_valuation_usd': valuation of the company
            'raised_amount_usd': amount of money raised in the deal (in thousands)

    """
    # If no IDs are specified, assume all funding rounds are needed
    if funding_round_ids is None:
        funding_round_ids = funding_rounds.id.unique()

    return (
        funding_rounds.query("id in @funding_round_ids")
        # More informative column names
        .rename(columns={"id": "funding_round_id"})
        # Remove really old funding rounds
        .query(f"announced_on > '{cutoff_year}'")
        .sort_values("announced_on")
        .assign(
            # Convert investment amounts to thousands
            raised_amount=lambda df: df["raised_amount"] / 1e+3,
            raised_amount_usd=lambda df: df["raised_amount_usd"] / 1e+3,
            # Convert date strings to datetimes
            announced_on_date=lambda df: pd.to_datetime(df["announced_on"]),
        )
        # Get years from dates
        .assign(year=lambda df: df["announced_on_date"].apply(lambda x: x.year))
        # Convert USD currency to GBP
        .pipe(
            convert_currency,
            date_column="announced_on_date",
            amount_column="raised_amount",
            currency_column="raised_amount_currency_code",
            converted_column="raised_amount_gbp",
            target_currency="GBP",
        )
    )


def _enrich_total_funding_gbp(
    organisations: pd.DataFrame,
    funding_rounds_enriched_df: pd.DataFrame,
    date_start: int = None,
    date_end:  str = None,
):
    """
    Get the total funding in GBP for a set of organisations

    Args:
        funding_rounds_enriched_df: Dataframe with enriched funding rounds
        date_start: Earliest date for the funding rounds to include, in the format 'YYYY-MM-DD'
        date_end: Latest date for the funding rounds to include, in the format 'YYYY-MM-DD'
    
    Returns:
        Dataframe with columns 'org_id' and 'total_funding_gbp'. Organisations that have no funding
        in the specified time period will not be included in the dataframe
    """
    # If no IDs are specified, assume all organisations are needed
    organisation_ids = funding_rounds_enriched_df.org_id.unique()

    # Determine the dates to include
    date_start = date_start or funding_rounds_enriched_df.announced_on.min()
    date_end = date_end or funding_rounds_enriched_df.announced_on.max()

    funding_rounds_df = (
        funding_rounds_enriched_df
        .loc[funding_rounds_enriched_df['announced_on'].between(date_start, date_end, inclusive='both')]
        .query("org_id in @organisation_ids")
        .groupby("org_id")
        .agg(
            total_funding_gbp=("raised_amount_gbp", "sum"),
            num_rounds=("funding_round_id", "count"),
        )
        .reindex(organisation_ids)
        .reset_index()
        .fillna(0)        
    )

    return (
        organisations
        .merge(
            funding_rounds_df,
            how = 'left',
            left_on='id',
            right_on='org_id'
        )
        .drop(columns='org_id')
    )   


In [40]:
def _process_max_employees(count_range: str) -> int:
    """
    Convert the employee count range to a single (highest) number

    Args:
        count_range: A string with the employee count range, eg, '51-100', '10000+'

    Returns:
        A single integer with the maximum number of employees
    """
    if count_range is None:
        return 0
    elif type(count_range) == str:
        return int(count_range.split('-')[-1].replace('+', ''))
    else:
        raise ValueError(f"Unexpected value for employee count: {count_range}")


def _enrich_investment_opportunity(
    organisations_enriched: pd.DataFrame
) -> pd.DataFrame:
    """
    """
    # Check for valid employee count and country
    valid_count_and_country = (
        (organisations_enriched.employee_count_max <= 100) 
        & (organisations_enriched.country_code == 'GBR')         
    )
    # Investment opportunities tags
    potential_investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp < 1e+3)
    )
    investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp >= 1e+3)
        & (organisations_enriched.total_funding_gbp <= 5e+3)
    )
    
    return (
        organisations_enriched
        .assign(
            potential_investment_opp = potential_investment_opp,
            investment_opp = investment_opp
        )
    )    

def _process_org_investments(
    
)

def _enrich_smart_money(
    organisations: pd.DataFrame,
    funding_rounds: pd.DataFrame,
    investments: pd.DataFrame,
    investors: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    return (
        organisations
        .assign(
            smart_money = ().investor_types.str.contains('|'.join(SMART_MONEY_TYPES)))
            )
        )
    )


def enrich_organisations(
    organisations: pd.DataFrame,
    funding_rounds_enriched_df: pd.DataFrame,
    organisation_ids: Iterator[str] = None,
) -> pd.DataFrame:
    """
    """
     # If no IDs are specified, assume all organisations are needed
    organisation_ids = organisation_ids or organisations.id.unique()

    organisations_enriched = (
        organisations
        .query("id in @organisation_ids")
        .assign(employee_count_max = lambda df: df['employee_count'].apply(_process_max_employees))
        .pipe(
            _enrich_total_funding_gbp,
            funding_rounds_enriched_df=funding_rounds_enriched_df
        )
        .pipe(_enrich_investment_opportunity)
        .pipe(_enrich_smart_money)
    )
    return organisations_enriched




In [ ]:
    def get_funding_round_investors(self, funding_rounds: pd.DataFrame) -> pd.DataFrame:
        """
        Gets the investors involved in the specified funding rounds

        Args:
            Dataframe with organisations specified by 'org_id' and 'name', and
            all their funding rounds (deals).

        Returns:
            Dataframe with extra columns specifying investment round details:
                'funding_round_name': Name of the funding round
                'investor_name': Name of the investor organisation
                'investor_id' Crunchbase organisation identifier
                'investor_type': Specifies if investor is a person or an organisation
                'is_lead_investor': Specifies whether the organisation is leading the round (value is 1 in that case)

        """
        return funding_rounds.merge(
            (
                self.cb_investments[
                    [
                        "funding_round_id",
                        "funding_round_name",
                        "investor_name",
                        "id",
                        "investor_type",
                        "is_lead_investor",
                    ]
                ].rename(columns={"id": "investor_id"})
            ),
            on="funding_round_id",
            how="left",
        ).assign(
            raised_amount=lambda x: x.raised_amount.fillna(0),
            raised_amount_usd=lambda x: x.raised_amount_usd.fillna(0),
        )

    def get_organisation_investors(
        self, cb_organisations: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Gets the investors and corresponding funding rounds for the specified organisations

        Args:
            cb_organisations: Data frame that must have a columns with crunchbase
                organisation ids and names

        Returns:
             Dataframe with extra columns specifying investors and the corresponding investment rounds
             (see docs for get_funding_rounds and get_funding_round_investors for more information)

        """
        return pipe(
            cb_organisations, self.get_funding_rounds, self.get_funding_round_investors
        )

In [8]:
from src.utils import s3
from src.utils import google
import importlib
importlib.reload(s3);
importlib.reload(google);

In [10]:
google.access_google_sheet(
    "1-Sl2kglCa09r2Lg9CsGEKocc3m8pukOwwrbkQk7706I",
    "ASF",
)

,Subcategory,Keywords,Comments
Decarbonisation,Built environment,"decarbonisation, build",
Decarbonisation,Built environment,"decarbonisation, built",
Decarbonisation,Built environment,"decarbonisation, built environment",
Decarbonisation,Built environment,"decarbonisation, home",
Decarbonisation,Built environment,"decarbonisation, house",
...,...,...,...
Renewables,Solar,solar energy,
Renewables,Solar,solar panel,
Renewables,Wind,wind energy,
Renewables,Wind,wind farm,


In [23]:
import dotenv
import os
dotenv.load_dotenv()



In [22]:
os.getenv("SHEET_ID_INVESTORS")

In [18]:
smart_money_df

,Crunchbase URL,Comments
Nesta Impact Investments,https://www.crunchbase.com/organization/nesta-...,


In [170]:
S3 = s3.s3_client()

In [ ]:
S3.download_file()

In [152]:
counts = sorted(list(cb_organisations.employee_count.dropna().unique()))
# split, take the largest number and remove non-numeric characters
[int(c.split('-')[-1].replace('+', '')) for c in counts]

[10, 10000, 5000, 250, 50, 500, 10000, 1000, 100]

In [ ]:
employee_count
country_code
total_funding_gbp

In [137]:
cb_organisations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3302538 entries, 0 to 3302537
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   id                           object 
 1   name                         object 
 2   type                         object 
 3   permalink                    object 
 4   cb_url                       object 
 5   rank                         float64
 6   created_at                   object 
 7   updated_at                   object 
 8   legal_name                   object 
 9   roles                        object 
 10  domain                       object 
 11  homepage_url                 object 
 12  country_code                 object 
 13  state_code                   object 
 14  region                       object 
 15  city                         object 
 16  address                      object 
 17  postal_code                  object 
 18  status                       object 
 19  

In [32]:
# Get funding rounds enriched
funding_rounds_enriched_df = enrich_cb_funding_rounds(funding_rounds)

In [131]:
# _process_total_funding_gbp(funding_rounds_enriched_df, None, '2022-01-01')

In [41]:
orgs_enriched = enrich_organisations(cb_organisations.head(10), funding_rounds_enriched_df)

AttributeError: 'DataFrame' object has no attribute 'investor_types'

In [34]:
orgs_enriched

,id,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,...,logo_url,alias1,alias2,alias3,primary_role,num_exits,total_funding_gbp,employee_count_max,potential_investment_opp,investment_opp
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,160064.0,2007-05-25 13:51:27,2023-08-01 00:01:37,None,company,...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,company,NaN,2.060731e+04,100,False,False
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,132132.0,2007-05-26 02:30:28,2023-10-19 07:46:32,Zoho Corporation Pvt. Ltd.,"investor,company",...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,company,2.0,0.000000e+00,10000,False,False
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,25964.0,2007-05-26 03:03:23,2024-01-26 08:06:40,"Digg Holdings, LLC",company,...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,company,NaN,2.715502e+04,100,False,False
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,42025.0,2007-05-26 03:21:34,2023-08-30 02:27:06,None,"investor,company",...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,investor,62.0,0.000000e+00,250,False,False
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,organization,facebook,https://www.crunchbase.com/organization/facebook,55.0,2007-05-26 04:22:15,2023-11-27 12:05:27,"Meta Platforms, Inc.","investor,company",...,https://images.crunchbase.com/image/upload/t_c...,Facebook,"Facebook, Inc.",FB,company,1.0,1.727092e+07,10000,False,False
5,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,organization,trinity-ventures,https://www.crunchbase.com/organization/trinit...,54164.0,2007-05-26 20:13:00,2023-08-30 23:04:05,None,"investor,company",...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,investor,144.0,0.000000e+00,50,False,False
6,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,organization,accel,https://www.crunchbase.com/organization/accel,6989.0,2007-05-26 20:27:39,2023-12-27 00:09:11,Accel Partners LP,"investor,company",...,https://images.crunchbase.com/image/upload/t_c...,Accel Partners,None,None,investor,370.0,0.000000e+00,250,False,False
7,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,organization,meritech-capital-partners,https://www.crunchbase.com/organization/merite...,53714.0,2007-05-26 20:29:59,2023-08-30 02:30:19,None,"investor,company",...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,investor,130.0,0.000000e+00,50,False,False
8,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,organization,omnidrive,https://www.crunchbase.com/organization/omnidrive,733451.0,2007-05-27 10:25:32,2019-10-03 06:30:53,None,company,...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,company,NaN,4.064633e+02,0,False,False
9,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,organization,geni,https://www.crunchbase.com/organization/geni,205852.0,2007-06-01 02:52:34,2020-12-24 11:21:45,None,company,...,https://images.crunchbase.com/image/upload/t_c...,None,None,None,company,NaN,8.525335e+03,50,False,False


In [ ]:
def investment_opportunity(
    organisations_enriched: pd.DataFrame,
)

In [ ]:



convert_currency(
    funding_df=funding_rounds.head(3),
    date_column="announced_on",
    amount_column="raised_amount",
    currency_column="raised_amount_currency_code",
    converted_column="raised_amount_gbp",
    target_currency="GBP",
)

In [ ]:
    def convert_deal_currency_to_gbp(
        funding: pd.DataFrame,
        date_column: str = "announced_on_date",
        amount_column: str = "raised_amount",
        usd_amount_column: str = "raised_amount_usd",
        converted_column: str = "raised_amount_gbp",